In [1]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory;pip install -e ".[torch,metrics]"
!pip install bitsandbytes>=0.39.0

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 15353, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 15353 (delta 191), reused 262 (delta 167), pack-reused 15050
Receiving objects: 100% (15353/15353), 221.76 MiB | 28.02 MiB/s, done.
Resolving deltas: 100% (11224/11224), done.
Obtaining file:///kaggle/working/LLaMA-Factory
  Installing build dependencies ... - \ | / done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... - done
  Preparing editable metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json

file_path = '/kaggle/working/LLaMA-Factory/data/dataset_info.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# configure the MedQA and PubMedQA dataset information 
new_data = {
    "MedQA_train": {"file_name": "MedQA/train.json"},
    "MedQA_test": {"file_name": "MedQA/test.json"},
    "PubMedQA_pqal_train": {"file_name": "PubMedQA/pqal_train_set.json"},
    "PubMedQA_pqal_test": {"file_name": "PubMedQA/pqal_test_set.json"}
}

# update information in dataset_info.json file
data.update(new_data)
with open(file_path, 'w') as file:
    json.dump(data, file, indent=4)

In [3]:
!ls /kaggle/input/

medqa-pubmedqa


In [4]:
# create MedQA and PubMedQA directories
!mkdir -p /kaggle/working/LLaMA-Factory/data/MedQA
!mkdir -p /kaggle/working/LLaMA-Factory/data/PubMedQA

# copy MedQA dataset file
!cp /kaggle/input/medqa-pubmedqa/MedQA/train.json /kaggle/working/LLaMA-Factory/data/MedQA/
!cp /kaggle/input/medqa-pubmedqa/MedQA/test.json /kaggle/working/LLaMA-Factory/data/MedQA/

# copy PubMedQA dataset file
!cp /kaggle/input/medqa-pubmedqa/PubMedQA/pqal_train_set.json /kaggle/working/LLaMA-Factory/data/PubMedQA/
!cp /kaggle/input/medqa-pubmedqa/PubMedQA/pqal_test_set.json /kaggle/working/LLaMA-Factory/data/PubMedQA/

In [5]:
%cd LLaMA-Factory

/kaggle/working/LLaMA-Factory


In [6]:
# install packages for quantizaiton
!pip install auto_gptq>=0.5.0
!pip install optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.7/424.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00


max_example = 500, lr = 1e-05

In [7]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 1e-05 \
    --num_train_epochs 3.0 \
    --max_samples 500 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-25 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 10 \
    --per_device_eval_batch_size 2 

2024-07-13 04:17:17.992202: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 04:17:17.992324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 04:17:18.127092: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 04:17:34 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
tokenizer_config.json: 100%|███████████████| 1.29k/1.29k [00:00<00:00, 8.30MB/s]
vocab.json: 100%|██████████████████████████| 2.78M/2.78M [00:00<00:00, 7.15MB/s]
merges.txt: 100%|██████████████████████████

In [8]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\eval_2024-07-09-00-25-34 \
    --do_predict True \
    --adapter_name_or_path saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-25

2024-07-13 04:31:53.493148: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 04:31:53.493210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 04:31:53.494819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 04:32:02 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2108] 2024-07-13 04:32:02,259 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct/snapshots/c291d6fce4804a1d39305f388dd32897d1f7a

max_example = 500, lr = 1e-04

In [9]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 1e-04 \
    --num_train_epochs 3.0 \
    --max_samples 500 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-26 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 10 \
    --per_device_eval_batch_size 2 

2024-07-13 04:34:48.242886: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 04:34:48.242958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 04:34:48.244482: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 04:34:56 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2108] 2024-07-13 04:34:57,047 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct/snapshots/c291d6fce4804a1d39305f388dd3

In [10]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\eval_2024-07-09-00-25-35 \
    --do_predict True \
    --adapter_name_or_path saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-26

2024-07-13 04:49:03.236834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 04:49:03.236890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 04:49:03.238445: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 04:49:11 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2108] 2024-07-13 04:49:12,016 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct/snapshots/c291d6fce4804a1d39305f388dd32897d1f7a

max_example = 500, lr = 5e-04

In [11]:
!llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_train,PubMedQA_pqal_train \
    --cutoff_len 1024 \
    --learning_rate 5e-04 \
    --num_train_epochs 3.0 \
    --max_samples 500 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 1000 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --packing False \
    --report_to none \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-27 \
    --fp16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --include_num_input_tokens_seen True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 10 \
    --per_device_eval_batch_size 2 

2024-07-13 04:51:31.316709: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 04:51:31.316771: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 04:51:31.318274: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 04:51:39 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.float16
[INFO|tokenization_utils_base.py:2108] 2024-07-13 04:51:40,100 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct/snapshots/c291d6fce4804a1d39305f388dd3

In [12]:
!llamafactory-cli train \
    --stage sft \
    --model_name_or_path Qwen/Qwen2-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset MedQA_test,PubMedQA_pqal_test \
    --cutoff_len 1024 \
    --max_samples 100 \
    --per_device_eval_batch_size 2 \
    --predict_with_generate True \
    --max_new_tokens 512 \
    --top_p 0.7 \
    --temperature 0.95 \
    --output_dir saves\Qwen2-0.5B-Instruct\lora\eval_2024-07-09-00-25-36 \
    --do_predict True \
    --adapter_name_or_path saves\Qwen2-0.5B-Instruct\lora\train_2024-07-08-20-40-27

2024-07-13 05:05:44.875975: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 05:05:44.876032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 05:05:44.877694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
07/13/2024 05:05:53 - INFO - llamafactory.hparams.parser - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: None
[INFO|tokenization_utils_base.py:2108] 2024-07-13 05:05:53,639 >> loading file vocab.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-0.5B-Instruct/snapshots/c291d6fce4804a1d39305f388dd32897d1f7a